In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import time
import hashlib
from neo4j import GraphDatabase
import json
import re


In [2]:
db_host = "bolt://localhost:7687"
password = "1234qwer"


In [53]:
api_characters = pd.read_csv("./Datasets/api_character_list.csv")
scraping_characters = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_relations.csv")
api_comics = pd.read_csv("./Datasets/merged.csv")


In [ ]:
scraping_characters.head(20)

In [ ]:
api_characters.head(20)

Add characters nodes with variant from API

In [50]:
def add_character_node(tx, name, character_id, character_description):
  processed_name = name.replace('"', " ").replace('"'," ")
  nome_senza_parentesi = re.sub("[\(\[].*?[\)\]]", "", processed_name.strip())
  query_string = 'MERGE (c:character{name: "%s"})' %(nome_senza_parentesi.replace("'", " ").lower().strip())
  tx.run(query_string)
  tx.run(
      "MERGE (n:character_variant {name: '%s', character_id: %s, character_description: '%s'})" %(name.replace("'", ' ').replace('"', " ").replace("-", " "), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ")))
  query_variant = 'MATCH (a:character{name:"%s"}),(b:character_variant {name: "%s", character_id: %s, character_description: "%s"}) MERGE (b)-[r:variante_di]->(a)' % (
      nome_senza_parentesi.replace("'", " ").lower().strip(), name.replace("'", ' ').replace('"', " ").replace("-", " ").strip(), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ").strip(),)
  tx.run(query_variant)


In [72]:
def add_comic_node_and_link(tx, comic_title, comic_id, character_id, comic_description):
  comic_description = str(comic_description).replace("'", " ").replace('"',"").replace("-"," ").strip()
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s, comic_description: "%s"})' % (
      comic_title, comic_id, comic_description)
  create_link_query = "MATCH (a:character_variant),(b:comic) WHERE a.character_id = %s AND b.comic_id = %s MERGE (a)-[r:Presente]->(b)" % (
      character_id, comic_id)
  # print(create_link_query)
  node_creation_result = tx.run(create_comic_node_query)
  link_creation_result = tx.run(create_link_query)


In [73]:
def add_comic_node(tx, comic_title,comic_id, comic_description):
  comic_description = str(comic_description).replace(
      "'", " ").replace('"', "").replace("-", " ").strip()
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s, comic_description: "%s"})' % (
      comic_title, comic_id, comic_description)
  node_creation_result = tx.run(create_comic_node_query)


In [74]:
driver = GraphDatabase.driver(db_host, auth=("neo4j",password))

In [65]:

for idx, row in tqdm(api_characters.iterrows(), total=api_characters.shape[0]):
    with driver.session() as session:
        try:
            session.write_transaction(
                add_character_node, row['name'], row["id"], row["description"])
        except Exception as e:
            print(e
            )

100%|██████████| 1559/1559 [01:02<00:00, 24.76it/s]


In [75]:
for idx, row in tqdm(api_comics.iterrows(), total=api_comics.shape[0]):
    string = row["characters"]
    string = string.replace("'", '"')
    characters = json.loads(string)
    if characters["items"]:
        for character in characters["items"]:
            with driver.session() as session:
                try:
                    character_id = character["resourceURI"].split("/")
                    character_id = character_id[len(character_id) -1]
                    session.write_transaction(
                            add_comic_node_and_link, row['title'], row["id"], character_id,  row["description"])
                except Exception as e:
                    print(e)
    else:
        with driver.session() as session:
            try:
                session.write_transaction(
                    add_comic_node, row['title'], row["id"],  row["description"])
            except Exception as e:
                print(e)
    


 12%|█▏        | 5939/50549 [05:15<39:28, 18.83it/s]  


JSONDecodeError: Expecting ',' delimiter: line 1 column 193 (char 192)